In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
CMIP5ds = xr.open_dataset('Data/CMIP5_time_series_smoothed.nc')
CMIP6ds = xr.open_dataset('Data/CMIP6_time_series_smoothed.nc')

In [8]:
# Loop over reference periods and scenarios:
ttest_results_total = []
for reference_period in [[1900, 1950], [1986, 2005], [1995, 2014]]:
    
    ref_st, ref_end = reference_period[0], reference_period[1]
    CMIP5ds = CMIP5ds - CMIP5ds.sel(time=slice(ref_st,ref_end)).mean(dim='time')
    CMIP6ds = CMIP6ds - CMIP6ds.sel(time=slice(ref_st,ref_end)).mean(dim='time')
    
    ref_results = []
    print(f'REFERENCE PERIOD: {ref_st}-{ref_end}')
    for sces in range(3):
        CMIP5 = CMIP5ds.isel(scenario=sces)
        CMIP6 = CMIP6ds.isel(scenario=sces)

        CMIP5 = CMIP5['zos'].dropna('model','any')
        CMIP6 = CMIP6['zos'].dropna('model','any')

        print('Result', CMIP5.scenario.values, CMIP6.scenario.values)
        diff_CMIP5 = CMIP5.sel(time=slice(2081, 2101)).mean(dim='time') - CMIP5.sel(time=slice(ref_st, ref_end)).mean(dim='time')
        print(f'Mean difference CMIP5: {np.mean(diff_CMIP5.values):.2f} cm')
        print('Number of models:', len(diff_CMIP5.values))

        diff_CMIP6 = CMIP6.sel(time=slice(2081, 2101)).mean(dim='time') - CMIP6.sel(time=slice(ref_st, ref_end)).mean(dim='time')
        print(f'Mean difference CMIP6: {np.mean(diff_CMIP6.values):.2f} cm')
        print('Number of models:', len(diff_CMIP6.values),'\n')

        print(f'Difference between the two means: {(np.mean(diff_CMIP5.values)-np.mean(diff_CMIP6.values)):.2f} cm' )
        print(f'p-value: {stats.ttest_ind(diff_CMIP5, diff_CMIP6)[1]:.4f}')
        print(stats.ttest_ind(diff_CMIP5, diff_CMIP6),'\n')

        results = [np.mean(diff_CMIP5.values),np.mean(diff_CMIP6.values),np.mean(diff_CMIP6.values)-np.mean(diff_CMIP5.values),stats.ttest_ind(diff_CMIP5, diff_CMIP6)[1]]

        ref_results.append(results)
    
    ttest_results_total.append(ref_results)

REFERENCE PERIOD: 1900-1950
Result rcp26 ssp126
Mean difference CMIP5: 6.00 cm
Number of models: 20
Mean difference CMIP6: 12.56 cm
Number of models: 34 

Difference between the two means: -6.56 cm
p-value: 0.0148
Ttest_indResult(statistic=-2.5213197466092847, pvalue=0.014795441424589037) 

Result rcp45 ssp245
Mean difference CMIP5: 8.97 cm
Number of models: 29
Mean difference CMIP6: 15.33 cm
Number of models: 34 

Difference between the two means: -6.36 cm
p-value: 0.0079
Ttest_indResult(statistic=-2.7462348193941932, pvalue=0.007911972935539585) 

Result rcp85 ssp585
Mean difference CMIP5: 14.69 cm
Number of models: 30
Mean difference CMIP6: 19.04 cm
Number of models: 37 

Difference between the two means: -4.35 cm
p-value: 0.0789
Ttest_indResult(statistic=-1.7852151746261564, pvalue=0.07889198823510414) 

REFERENCE PERIOD: 1986-2005
Result rcp26 ssp126
Mean difference CMIP5: 4.63 cm
Number of models: 20
Mean difference CMIP6: 12.53 cm
Number of models: 34 

Difference between the tw

In [16]:
d = {'index': ['mean CMIP5', 'mean CMIP6', 'difference CMIP6 - CMIP5', 'p-value'], '1900-1950': ttest_results_total[0][0], '1986-2005':ttest_results_total[1][0], '1995-2014':ttest_results_total[2][0]}
df_26 =  pd.DataFrame(data=d).set_index('index')  
print('Results for RCP2.6/SSP1-2.6:')
df_26.round(3)

Results for RCP2.6/SSP1-2.6:


,1900-1950,1986-2005,1995-2014
index,,,
mean CMIP5,6.004,4.629,3.962
mean CMIP6,12.563,12.527,11.131
difference CMIP6 - CMIP5,6.560,7.898,7.169
p-value,0.015,0.003,0.004


In [17]:
d = {'index': ['mean CMIP5', 'mean CMIP6', 'difference CMIP6 - CMIP5', 'p-value'], '1900-1950': ttest_results_total[0][1], '1986-2005':ttest_results_total[1][1], '1995-2014':ttest_results_total[2][1]}
df_45 =  pd.DataFrame(data=d).set_index('index') 
print('Results for RCP4.5/SSP2-4.5:')
df_45.round(3)

Results for RCP4.5/SSP2-4.5:


,1900-1950,1986-2005,1995-2014
index,,,
mean CMIP5,8.974,7.774,7.127
mean CMIP6,15.333,15.336,13.957
difference CMIP6 - CMIP5,6.358,7.562,6.830
p-value,0.008,0.001,0.002


In [18]:
d = {'index': ['mean CMIP5', 'mean CMIP6', 'difference CMIP6 - CMIP5', 'p-value'], '1900-1950': ttest_results_total[0][2], '1986-2005':ttest_results_total[1][2], '1995-2014':ttest_results_total[2][2]}
df_85 =  pd.DataFrame(data=d).set_index('index') 
print('Results for RCP8.5/SSP5-8.5:')
df_85.round(3)

Results for RCP8.5/SSP5-8.5:


,1900-1950,1986-2005,1995-2014
index,,,
mean CMIP5,14.690,13.574,12.871
mean CMIP6,19.040,19.023,17.689
difference CMIP6 - CMIP5,4.351,5.449,4.818
p-value,0.079,0.022,0.034
